In [1]:
# Keras fix
import tensorflow
from tensorflow.python.ops import control_flow_ops
tensorflow.python.control_flow_ops = control_flow_ops

from keras.models import load_model
from nl.text_gen import generate_from_model
from nl.utils import load_w2v_data, GenerateSamples, print_word_list, parse_text

Using TensorFlow backend.
/home/m20163692/anaconda3/envs/tf/lib/python3.5/site-packages/gensim/utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


In [2]:
print('Loading files...')

text_file = 'text_files/asimov'
model_file = 'models/asimov_model.h5'
mode = 'general_1'

# with tensorflow.device('/cpu:0'):
#     model = load_model(model_file)

model = load_model(model_file)

seq_len = model.input_shape[1]
input_dim = model.input_shape[2]

word2idx, idx2word, embeddings = load_w2v_data('w2v_embs/asimov/asimov_vocab.json',
                                               'w2v_embs/asimov/asimov_embeddings.npz')

Loading files...


In [3]:
# Random samples from text_file
n_samples = 3
n_words = 50
print_truth = True
n_truth_words = 30
temperatures = [0.25, 0.5, 0.75]

samples = GenerateSamples(word2idx, text_file, mode, 'dir', n_samples, n_words)

for idx, sample in enumerate(samples):
    print('-' * 80)
    print('SAMPLE %d:' % (idx + 1))
    print_word_list(sample[:seq_len])
    
    if print_truth:
        print('\nTRUTH:')
        print_word_list(sample[:seq_len + n_truth_words])
    
    for temperature in temperatures:
        print('\nGENERATED FROM MODEL (temperature = %f):' % temperature)
        model_sample = generate_from_model(sample, model, embeddings, idx2word,
                                           word2idx, n_words, temperature, mode=mode)
        print_word_list(model_sample)
    
    print('-' * 80)

--------------------------------------------------------------------------------
SAMPLE 1:
— siempre me ha parecido entender que gladia no era feliz en

TRUTH:
— siempre me ha parecido entender que gladia no era feliz en su planeta natal, que abandonó su mundo con alegría y que en ningún momento tuvo intención de regresar. no obstante, estoy de acuerdo contigo en que la

GENERATED FROM MODEL (temperature = 0.250000):
— siempre me ha parecido entender que gladia no era feliz en su mundo. 
 — ¿qué es lo que no te parece? 
 — ¿qué es lo que te hace, entonces? 
 — ¿qué es lo que no te parece? 
 — no lo sé. 
 — ¿y si no le

GENERATED FROM MODEL (temperature = 0.500000):
— siempre me ha parecido entender que gladia no era feliz en su vida. 
 — ¿qué es lo que quieres? 
 — si no es así, no lo sé. 
 — ¿y qué tiene que ver con el problema? 
 — en este momento, señor, no me lo he dicho. no

GENERATED FROM MODEL (temperature = 0.750000):
— siempre me ha parecido entender que gladia no era feliz en

In [3]:
# Using custom text
text = 'en el centro de nuestra galaxia comenzó'
n_words = 100
temperatures = [0.25, 0.5, 0.75]

print('SOURCE TEXT:\n%s' % text)

for temperature in temperatures:
    print('\nGENERATED FROM MODEL (temperature = %f):' % temperature)
    model_sample = generate_from_model(text, model, embeddings, idx2word, word2idx,
                                       n_words, temperature, truncating='post',
                                       custom_text=True, mode=mode)
    print_word_list(model_sample)

SOURCE TEXT:
en el centro de nuestra galaxia comenzó

GENERATED FROM MODEL (temperature = 0.250000):
en el centro de nuestra galaxia comenzó a estar de pie, y el otro, a un lado, se había dado cuenta de que el robot no era un robot, y el robot se la por la que se había ido, la más importante, el más pequeño de los mundos y el mismo los que han sido de la tierra, ¿verdad? 
 — sí, señor. 
 — ¿y si no es un robot? 
 — no, no lo sé. 
 — ¿y si no lo sea? 
 — ¿por qué

GENERATED FROM MODEL (temperature = 0.500000):
en el centro de nuestra galaxia comenzó la vez más a la vez. el doctor fastolfe se inclinó hacia atrás, y se fue de lo más. 
 — ¿y qué es lo que nos? 
 — no, no. no me — dijo, y añadió —: ¡no, no, no! en aquel momento... el que había estado en el otro extremo de la galaxia se hizo de nuevo la vida en la que se había dado cuenta, y después de todo, la respuesta de la palabra era la de un robot y un

GENERATED FROM MODEL (temperature = 0.750000):
en el centro de nuestra galaxia come

# Checking words

In [4]:
import nltk

tags_corpus = nltk.corpus.conll2002.tagged_words()
tags_dict = {word:tag for word, tag in tags_corpus[:25000]}

In [5]:
i = 0
www = []

for word, tag in tags_corpus[:25000]:
    word = word.lower()
    if i == 25:
        break
        
    if tag == 'PP':
        try:
            www.index(word)
        except:
            print(word)
            www.append(word)
            i += 1

lo
le
ellos
la
ellas
nos
nosotros
los
sí
él
les
yo
ello
ella


In [7]:
import numpy as np
from nl.text_gen import get_next_word_distribution

text = 'y si la verdad fuera'
temperature = 0.65
n_words = 300

t = parse_text(text, word2idx, mode)
tags = []
tagged_words = []

# def get_tag():
    
for i in range(n_words):
    last_word = t[-1]
    preds = get_next_word_distribution(t, model, embeddings, word2idx, temperature, mode=mode)
    probs = np.random.multinomial(5, preds, 1).reshape(-1)
    words = np.argsort(probs)[::-1]
    test = 0
    next_word = idx2word[words[test]]
    
    if last_word == next_word:
        test += 1
        next_word = words[test]
    
#     if 5-np.max(probs) < 2:
#         print(next_word, '-', 5-np.max(probs))
    
    success = -1
    tries = 0
    
    while(success != 0):
        success = 0
        
        if tries > 0:
            test += 1
#             print('-try-')

        next_word = idx2word[words[test]]
        
        try:
            this_tag = tags_dict[next_word]
        except:
            this_tag = None

        if this_tag:
            try :
                if tagged_words[-1] == tagged_words[-2] == next_word:
                    success -= 1
                    
                if tags[-1] == this_tag == 'DA':
                    success -= 1
                    
                if tags[-1] == this_tag == 'PP':
                    success -= 1
                    
                if tags[-1] == this_tag == 'VSI':
                    success -= 1
            except:
                None
                
        tries += 1
       
    tags.append(this_tag)
    
    if this_tag: 
        tagged_words.append(next_word)
    
    t.append(next_word)
    
print_word_list(t)

y si la verdad fuera de la tierra, ¿qué es lo que por. de que nosotros no sea el que le mundos, ¿verdad? 
 — sí, señor. 
 — ¿y si no es un robot? 
 — no, no lo sé. 
 — ¿del mismo modo que si todo el mundo puede que sea un robot, no puede ser que el doctor fastolfe sea un robot? 
 — no, no lo sé. 
 — ¿cómo te lo dijo? 
 — porque sólo me he dado cuenta de que era un robot. 
 — ¿por qué no? 
 — no lo sé. 
 — ¿por qué no? — dijo trevize —. si es necesario, eso no es lo que nos. 
 — ¿y si el doctor amadiro es un robot? 
 — es muy posible. 
 — ¿y qué es eso? 
 — no es. sólo hay un mundo que no tiene en cuenta la palabra, y no hay nada que la gente. — se interrumpió —. ¿es usted un robot? 
 — después de todo, usted no es un robot. 
 — ¿de qué manera? 
 — es muy difícil de ser. le soy un robot. 
 — ¿de qué manera? 
 — es muy posible que el doctor fastolfe no sea la. 
 — ¿por qué no lo aquí? 
 — porque es un robot. 
 — ¿de qué es lo que lo ha hecho? 
 — de un modo tan ser..., todo lo que tenía 